# Build the Neural Network

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Define the Class

We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of NeuralNetwork, and move it to the device, and print its structure.

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [67]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


## Model Layers

In [68]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [69]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

In [70]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [71]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0163, -0.1142, -0.1002, -0.2752,  0.0372,  0.2507, -0.2136, -0.0956,
          0.6841, -0.0771,  0.2445, -0.1894, -0.1110, -0.1607,  0.3963, -0.1063,
         -0.1648,  0.6957, -0.7221,  0.2078],
        [ 0.1827, -0.1453, -0.0160, -0.2527, -0.1458,  0.2749, -0.1396, -0.1885,
          0.1695,  0.0986,  0.1382, -0.1187,  0.2212, -0.1283,  0.2768, -0.2533,
         -0.1943,  0.3771, -0.7239,  0.4517],
        [-0.1598,  0.2498, -0.0569, -0.5171, -0.4758,  0.3970, -0.0515,  0.1608,
          0.4667,  0.1721,  0.5978, -0.0513, -0.2781, -0.0964,  0.1790, -0.0430,
         -0.0643,  0.6364, -0.5257,  0.2620]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0372, 0.2507, 0.0000, 0.0000, 0.6841,
         0.0000, 0.2445, 0.0000, 0.0000, 0.0000, 0.3963, 0.0000, 0.0000, 0.6957,
         0.0000, 0.2078],
        [0.1827, 0.0000, 0.0000, 0.0000, 0.0000, 0.2749, 0.0000, 0.0000, 0.1695,
         0.0986, 0.1382, 0.0000, 0.2212, 0.0000, 0.27

### nn.Sequential

nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [72]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

In [77]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.1277, 0.0895, 0.1114, 0.1030, 0.0869, 0.0725, 0.1035, 0.0963, 0.1190,
         0.0902],
        [0.1224, 0.0779, 0.1129, 0.0996, 0.0849, 0.0831, 0.1120, 0.0920, 0.1240,
         0.0912],
        [0.1332, 0.0926, 0.1121, 0.1139, 0.0866, 0.0751, 0.0911, 0.0989, 0.1037,
         0.0929]], grad_fn=<SoftmaxBackward0>)

### Model Parameters

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [78]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0341, -0.0198, -0.0196,  ...,  0.0303, -0.0189, -0.0156],
        [ 0.0036, -0.0140,  0.0187,  ..., -0.0021,  0.0054, -0.0038]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0241, -0.0303], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0245, -0.0091, -0.0396,  ...,  0.0071,  0.0057, -0.0291],
        [ 0.0413, -0.0157, -0.0317,  ...,  0.0191, -0.0283,  0.0148]],
       device='cuda:0', grad_fn=<Sl